In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tfv1
tf.__version__

In [ ]:
s = tf.constant(2)
m = tf.constant([[1,2],[3,4]])
m = tf.constant([1,2,3,4],shape=[2,2])

In [ ]:
g = tf.Graph()
with g.as_default():
    s = tf.constant(2)
    m = tf.constant([[1,2],[3,4]])
    mmul = s*m
g

In [ ]:
with tfv1.Session(graph=g) as sess:
    print(sess.run(mmul))

In [ ]:
g = tf.Graph()
with g.as_default():
    w = tf.Variable(tf.ones((2,2)))

In [ ]:
with tfv1.Session(graph=g) as sess:
    print(sess.run(w))
    # It will give u an error because you haven't initialize it.

In [ ]:
with tfv1.Session(graph=g) as sess:
    sess.run(tfv1.global_variables_initializer())
    print(sess.run(w))

In [ ]:
with tfv1.Session(graph=g) as sess:
    sess.run(tfv1.global_variables_initializer())
    print(sess.run(w))
    sess.run(w.assign(tf.zeros((2,2))))
    print(sess.run(w))

$\left( {\begin{array}{*{20}{c}}
{\rm{4}}&{\rm{5}}
\end{array}} \right){\rm{ = }}\left( {\begin{array}{*{20}{c}}
{\rm{1}}&{\rm{2}}
\end{array}} \right)\left( {\begin{array}{*{20}{c}}
{\rm{1}}&{\rm{1}}\\
{\rm{1}}&{\rm{1}}
\end{array}} \right){\rm{ + }}\left( {\begin{array}{*{20}{c}}
{\rm{1}}&{\rm{2}}
\end{array}} \right)$

In [ ]:
import numpy as np
tfv1.disable_eager_execution()
with g.as_default():
    node1 = tfv1.placeholder(tf.float32,shape = [1,2])
    node2 = tfv1.placeholder(tf.float32,shape = [1,2])
    w_linear = tf.matmul(node1,w) + node2
with tfv1.Session(graph=g) as sess:
    sess.run(tfv1.global_variables_initializer())
    print(sess.run(w))
    print(sess.run(w_linear,feed_dict={node1:np.matrix([1.0,2.0]),node2:np.matrix([1.0,2.0])}))

In [ ]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
  y = x**2
dy_dx = tape.gradient(y, x)
# dy = 2x * dx
with tfv1.Session() as sess:
    sess.run(tfv1.global_variables_initializer())
    print(sess.run(dy_dx))

In [ ]:
x0 = tf.Variable(3.0, name='x0')
x1 = tf.Variable(3.0, name='x1', trainable=False)
x2 = tf.Variable(2.0, name='x2') + 1.0
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2) 

# change x0 to more ?
grad = tape.gradient(y, [x0])

with tfv1.Session() as sess:
    sess.run(tfv1.global_variables_initializer())
    print(sess.run(grad))

In [ ]:
t1 = tf.constant(1)
t2 = tf.constant(2)
def f1(): return t1+t2
def f2(): return t1-t2
res = tf.cond(t1<t2,f1,f2)
with tfv1.Session() as sess:
    print(sess.run(res))

In [ ]:
t1 = tf.constant(1)
t2 = tf.constant(5)
body = (tf.add(t1,1),t2)
def cond(t1,t2): return t1<t2
def body(t1,t2): 
    t1 = tf.add(t1,1)
    return [t1,t2]
res = tf.while_loop(cond,body,[t1,t2])
with tfv1.Session() as sess:
    print(sess.run(res))

In [ ]:
## notice the difference between [t1,t2] and (t1,t2)
t1 = tf.constant(1)
t2 = tf.constant(5)
body = (tf.add(t1,1),t2)
def cond(t1,t2): return t1<t2
def body(t1,t2): 
    t1 = tf.add(t1,1)
    return (t1,t2)
res = tf.while_loop(cond,body,(t1,t2))
with tfv1.Session() as sess:
    print(sess.run(res))

  # Example of Linear Rregression

### Construct one iteration

### True model $Y = X\left( {\begin{array}{*{20}{c}}
1\\
5
\end{array}} \right) + \varepsilon ,\varepsilon  \sim N\left( {0,1} \right)$

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tfv1
tfv1.disable_eager_execution()
n_size = 5000
X = np.random.normal(0.0,1.0,size=(n_size ,2))
## beta real = (1.0,5.0)
Y = np.dot(X,np.matrix([[1.0],[5.0]])) + np.random.normal(0.0,1.0,size=(n_size ,1))

In [ ]:
beta = tf.Variable(np.array([[0.0],[0.0]]))
with tf.GradientTape() as tape:
    error = tf.subtract(Y,tf.matmul(X,beta))
    loss = tf.reduce_mean(tf.multiply(error,error))
grad = beta - 0.5*tape.gradient(loss,beta)
with tfv1.Session() as sess:
    sess.run(tfv1.global_variables_initializer())
    print(sess.run(grad))

In [ ]:
class Linear_Regression():
    tfv1.reset_default_graph()
    def __init__(self):
        self.beta = tf.Variable(np.array([[0.0],[0.0]]),dtype='float64',name='beta')
        
    def loss(self,x,y):
        error = tf.subtract(y,tf.matmul(x,self.beta))
        loss = tf.reduce_mean(tf.multiply(error,error))
        return loss
    
    def coefficient(self):
        with tfv1.Session() as sess:
            new_saver = tfv1.train.import_meta_graph('my-model.meta')
            new_saver.restore(sess,'my-model')
            return (sess.run(self.beta))
    
    def fit(self,x,y,gamma=0.5):
        maxit = 100
        with tf.GradientTape() as tape:
            tape.watch(self.beta)
            curent_loss = self.loss(x=X,y=Y)
        dw = tape.gradient(curent_loss,self.beta)
        self.beta = self.beta.assign_sub(gamma * dw)
        with tfv1.Session() as sess:
            sess.run(tfv1.global_variables_initializer())
            for it in range(maxit):
                print(sess.run(self.beta))
            print(sess.run(self.beta))
            saver = tfv1.train.Saver()
            saver.save(sess, 'my-model')
    
    def predict(self,x):
        y_predict = tf.matmul(x,self.beta)
        with tfv1.Session() as sess:
            new_saver = tfv1.train.import_meta_graph('my-model.meta')
            new_saver.restore(sess,'my-model')
            return sess.run(y_predict)

In [ ]:
model = Linear_Regression()

In [ ]:
model.fit(X,Y)

### Estimation ${{\hat \beta }^{\left( {100} \right)}} = ?$

In [ ]:
model.predict(x=np.ones((1,2)))

In [ ]:
model.coefficient()

# Example of sport data analysis

## Data description: 200 games from the English Premier League, we take the sentiment score from it.

In [ ]:
import pandas as pd
data_english = pd.read_csv('extract-betsentiment-com.csv')

In [ ]:
data_english

In [ ]:
data_english_prepare = data_english[['sentiment_score_st_a','sentiment_score_st_b','sentiment_score_lt_a','sentiment_score_lt_b','wgted_players_sentiment_a','wgted_players_sentiment_b','wgted_players_sentiment_st_a','wgted_players_sentiment_st_b','final_result']]
data_english_prepare

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_english_prepare.iloc[:,0:-1],data_english_prepare.iloc[:,-1])

In [ ]:
X_train.shape[0]

In [ ]:
X_test.shape[0]

In [ ]:
y_train_dm = pd.get_dummies(y_train)
y_test_dm = pd.get_dummies(y_test)

[<img src="https://www.tensorflow.org/images/custom_estimators/full_network.png">](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough)

In [ ]:
def create_train_model(hidden_nodes, num_iters):
    
    # Reset the graph
    tfv1.reset_default_graph()

    # Placeholders for input and output data
    X = tfv1.placeholder(shape=(150, 8), dtype=tf.float64, name='X')
    y = tfv1.placeholder(shape=(150, 3), dtype=tf.float64, name='y')

    # Variables for two group of weights between the three layers of the network
    W1 = tf.Variable(np.random.rand(8, hidden_nodes), dtype=tf.float64)
    W2 = tf.Variable(np.random.rand(hidden_nodes, 3), dtype=tf.float64)

    # Create the neural net graph
    A1 = tf.sigmoid(tf.matmul(X, W1))
    y_est = tf.sigmoid(tf.matmul(A1, W2))

    # Define a loss function
    deltas = tf.square(y_est - y)
    loss = tf.reduce_sum(deltas)

    # Define a train operation to minimize the loss
    optimizer = tfv1.train.GradientDescentOptimizer(0.005)
    train = optimizer.minimize(loss)

    # Initialize variables and run session
    init = tfv1.global_variables_initializer()
    sess = tfv1.Session()
    sess.run(init)

    # Go through num_iters iterations
    for i in range(num_iters):
        sess.run(train, feed_dict={X: X_train.values, y: y_train_dm.values})
        loss_plot[hidden_nodes].append(sess.run(loss, feed_dict={X: X_train.values, y: y_train_dm.values}))
        weights1 = sess.run(W1)
        weights2 = sess.run(W2)
        
    print("loss (hidden nodes: %d, iterations: %d): %.2f" % (hidden_nodes, num_iters, loss_plot[hidden_nodes][-1]))
    sess.close()
    return weights1, weights2

In [ ]:
import matplotlib.pyplot as plt
tfv1.disable_eager_execution()
num_hidden_nodes = [5, 10, 20]
loss_plot = {5: [], 10: [], 20: []}
weights1 = {5: None, 10: None, 20: None}
weights2 = {5: None, 10: None, 20: None}
num_iters = 2000

plt.figure(figsize=(12,8))
for hidden_nodes in num_hidden_nodes:
    weights1[hidden_nodes], weights2[hidden_nodes] = create_train_model(hidden_nodes, num_iters)
    plt.plot(range(num_iters), loss_plot[hidden_nodes], label="nn: 8-%d-3" % hidden_nodes)
    
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=12)

In [ ]:
class NN():
    tfv1.reset_default_graph()
    
    def __init__(self,hidden_nodes):
        self.W1 = tf.Variable(np.random.rand(8, hidden_nodes), dtype=tf.float64, name = 'W1')
        self.W2 = tf.Variable(np.random.rand(hidden_nodes, 3), dtype=tf.float64, name = 'W2')
        
    def loss(self,y1,y2):
        error = tf.subtract(y1,y2)
        loss = tf.reduce_mean(tf.multiply(error,error))
        return loss
    
    def coefficient(self):
        with tfv1.Session() as sess:
            new_saver = tfv1.train.import_meta_graph('my-model.meta')
            new_saver.restore(sess,'my-model')
            return (sess.run(self.W1),sess.run(self.W2))
    
    def fit(self,x_in,y_in,gamma=0.5):
        maxit = 2000
        X = tfv1.placeholder(shape=(150, 8), dtype=tf.float64, name='X')
        y = tfv1.placeholder(shape=(150, 3), dtype=tf.float64, name='y')
        A1 = tf.sigmoid(tf.matmul(X, self.W1))
        y_est = tf.sigmoid(tf.matmul(A1, self.W2))
        curent_loss = self.loss(y1=y_est,y2=y)
        optimizer = tfv1.train.GradientDescentOptimizer(0.005)
        train = optimizer.minimize(curent_loss)
        with tfv1.Session() as sess:
            sess.run(tfv1.global_variables_initializer())
            for it in range(maxit):
                sess.run(train, feed_dict={X: x_in, y: y_in})
                print(sess.run(curent_loss, feed_dict={X: x_in, y: y_in}))
                print(sess.run(self.W1))
            saver = tfv1.train.Saver()
            saver.save(sess, 'my-model')
    
    def predict(self,x):
        y_predict = tf.sigmoid(tf.matmul(tf.matmul(x,self.W1),self.W2))
        with tfv1.Session() as sess:
            new_saver = tfv1.train.import_meta_graph('my-model.meta')
            new_saver.restore(sess,'my-model')
            return sess.run(y_predict)

In [ ]:
model = NN(5)

In [ ]:
model.fit(X_train.values,y_train_dm)

In [ ]:
model.coefficient()

In [ ]:
model.predict(X_test.iloc[0,:].values.astype('float64').reshape(1,8))

In [ ]:
y_test.iloc[0]